## 📌 Introduction

This notebook analyzes student exam performance to understand how
demographic factors, parental background, and test preparation influence
academic outcomes.

The goal is to derive actionable insights using Pandas and NumPy.


In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input/students-performance-in-examst'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 📂 Dataset Overview

- Contains exam scores of students
- Subjects: Math, Reading, Writing
- Additional attributes: gender, parental education, test preparation, lunch type


In [12]:
import pandas as pd

df = pd.read_csv(
    "/kaggle/input/students-performance-in-exams/StudentsPerformance.csv"
)

df.head()



,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


##LEVEL 2-GROUPBY MASTERY

In [13]:
df[['math score','reading score','writing score']].mean()

math score       66.089
reading score    69.169
writing score    68.054
dtype: float64

Gender vs performance

In [14]:
df.groupby('gender')[['math score','reading score','writing score']].mean()


,math score,reading score,writing score
gender,,,
female,63.633205,72.608108,72.467181
male,68.728216,65.473029,63.311203


In [15]:
df.groupby('test preparation course')[['math score','reading score','writing score']].mean()

,math score,reading score,writing score
test preparation course,,,
completed,69.695531,73.893855,74.418994
none,64.077882,66.534268,64.504673


##LEVEL 3-FEATURE ENGINEERING

In [16]:
import numpy as np

df['math_level'] = np.where(df['math score'] >= 75, 'High', 'Low')
df['overall_avg'] = df[['math score','reading score','writing score']].mean(axis=1)


In [17]:
df.groupby('parental level of education')[['math score','reading score','writing score']].mean().sort_values('math score', ascending=False)


,math score,reading score,writing score
parental level of education,,,
master's degree,69.745763,75.372881,75.677966
bachelor's degree,69.389831,73.000000,73.381356
associate's degree,67.882883,70.927928,69.896396
some college,67.128319,69.460177,68.840708
some high school,63.497207,66.938547,64.888268
high school,62.137755,64.704082,62.448980


In [18]:
high_math = df[df['math score'] > 90]
high_math['gender']



34       male
104      male
114    female
121      male
149      male
165    female
171      male
179    female
233      male
263    female
286      male
306      male
451    female
458    female
469      male
501    female
503    female
521    female
539      male
546    female
562      male
566    female
571      male
594    female
612      male
618      male
623      male
625      male
685    female
689      male
710      male
712    female
717    female
719      male
736      male
779      male
784      male
815      male
846      male
855    female
864      male
886    female
903    female
916      male
919      male
934      male
950      male
957    female
962    female
979    female
Name: gender, dtype: object

In [19]:
Female students<50 in writing

SyntaxError: invalid syntax (314365156.py, line 1)

In [ ]:
female=df[(df['gender']=='female')&(df['writing score']<50)]
female.head()

Average score per subject by gender

In [ ]:
gender_avg=df.groupby('gender')[['math score','reading score','writing score']].mean()
gender_avg

In [ ]:
gender_avg.idxmax()


In [ ]:
avg=df.groupby('test preparation course')[['math score','reading score','writing score']].mean()
avg

In [ ]:
complete=avg.loc['completed']
non=avg.loc['none']
perc=((complete-non)*100)/non
perc

##LEVEL 4- CORRELATION

In [ ]:
parent_avg = df.groupby('parental level of education')[['math score','reading score','writing score']].mean()
parent_avg


In [ ]:
parent_avg.sort_values('math score', ascending=False)


In [ ]:
df['overall_score'] = df[['math score','reading score','writing score']].mean(axis=1)

df.groupby('race/ethnicity')['overall_score'].mean()


In [ ]:
prep_avg = df.groupby('test preparation course')[['math score','reading score','writing score']].mean()

prep_avg.loc['completed']
prep_avg.loc['none']


##LEVEL5

In [ ]:
df['overall_score'] = df[['math score','reading score','writing score']].mean(axis=1)
df[['math score','reading score','writing score','overall_score']].head()

In [ ]:
df['performance_level'] = np.where(
    df['overall_score'] >= 75, 'High',
    np.where(df['overall_score'] >= 50, 'Medium', 'Low')
)
df[['overall_score','performance_level']].head()


In [23]:

df['pass_status'] = np.where(
    (df['math score'] >= 40) &
    (df['reading score'] >= 40) &
    (df['writing score'] >= 40),
    'Pass', 'Fail'
)

df['pass_status'].value_counts()


pass_status
Pass    949
Fail     51
Name: count, dtype: int64

In [24]:
df.groupby('math score')['pass_status'] \
  .value_counts(normalize=True) * 100


math score  pass_status
0           Fail           100.0
8           Fail           100.0
18          Fail           100.0
19          Fail           100.0
22          Fail           100.0
                           ...  
96          Pass           100.0
97          Pass           100.0
98          Pass           100.0
99          Pass           100.0
100         Pass           100.0
Name: proportion, Length: 90, dtype: float64

In [25]:
df[df['pass_status'] == 'Fail'] \
  .groupby(['gender','test preparation course']) \
  .size()


gender  test preparation course
female  completed                   5
        none                       26
male    completed                   2
        none                       18
dtype: int64